# Plot paired and unpaired synapses

This notebook assumes that all of the studies are orginized into a standard Python data structure. All of the plots are done using plot.ly, so you should look at the appropriate documentation for that if you want to change them.

The variable `study_list` constains a list of studies, where each study is represented as a pyton dictionary. The keys in the dictionary currently include the study name, the subject id, the type of study (aversion, control), and if the fish learned or not. 

The python command `list(study_list[0])` will return a list of the current values that are available for a study.

The study dictionary may also contain a set of calculated synapse pairings.  Each pair is represented as a python dictionary, whose key is the radius with which the pairing was determined, and whose value is a PANDA, with the synapse positions of the pair. Each PANDA has included in it a set of useful values from the study, including the study id, the type of study, the radius for the pair, etc.  Pairs are also transformed using the alignment points to put them into a common coordinate space.  The sets of pairs provided are:

* PairedBefore
* PairedAfter
* UnpairedBefore
* UnpairedAfter
* AlignedPairedBefore
* AlignedPairedAfter
* AlignedUnpairedBefore
* AlignedUnpairedAfter

In [ ]:
# Load up all of the libraries we need
import matplotlib
from matplotlib import cm
import random

from  collections import OrderedDict
import synapse_plot_utils as sp
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

<H2> Useful Global Variables

In [ ]:
study_list = ['SynStd6025','SynStd8013','SynStd8071','SynStd6791','SynStd7951','SynStd7949','SynStd6493','SynStd7943',
          'SynStd6479','SynStd6481','SynStd6485','SynStd6483', 'SynStd6477', 'SynStdR5Y',
          'SynStdJK6','SynStdJKP','SynStdJKE','SynStdJJY','SynStdJJP','SynStdJJE','SynStdJJ6',
          'SynStdRVA','SynStdRVE','SynStdRVJ','SynStdRVP','SynStdRVT']

syn_pair_radii = (4, 300.0,) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold


# Load Data

You have two choices, you can reload data from a previous notebook run, or regenerate all of the pairs with data from the catalog. If there have been new studies added, or any data has changed, you must regenerate all the pairs. In general, you should probably only use the save and 

<h3> Restore data from previous run

In [ ]:
dumpfile = 'pairs-dump.pkl'
paired_studies = sp.restore_studies(dumpfile)

<H3> Get Study Pairs from Catalog

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [ ]:
dumpfile = 'pairs-dump.pkl'

all_studies = sp.get_studies()

learners = sp.group_studies(all_studies, group='Type')['learner']
nonlearners = sp.group_studies(all_studies, group='Type')['nonlearner']
aligned = sp.group_studies(all_studies, group='Aligned')[True]

print('{0} learners'.format(len(learners)))
print('{0} nonlearners'.format(len(nonlearners)))
print('{0} aligned'.format(len(aligned)))

# Lets just get the learners and nonlearners for now.
paired_studies = [  s for s in all_studies if s['Study'] in study_list]

sp.compute_pairs(paired_studies, syn_pair_radii)

dumpfile = 'pairs-dump.pkl'

sp.dump_studies(paired_studies, dumpfile)


<H1> Compute some basic Statistics

In [ ]:
# Set what radius/studies you want to have....

cols = ['ID', 'Type', 'Radius', 'Unpaired Before', 'Unpaired After','Paired Before','Paired After']
c1, c2, c3, c4, c5, c6, c7 = [],[],[],[],[],[],[]

syn_pair_radii = list(paired_studies[0]['UnpairedBefore'])

for s in paired_studies:
    for r in syn_pair_radii:
        c1.append(s['Study'])
        c2.append(s['Type'])
        c3.append(r)
        c4.append(len(s['UnpairedBefore'][r]['Data'].index))
        c5.append(len(s['UnpairedAfter'][r]['Data'].index))
        c6.append(len(s['PairedBefore'][r]['Data'].index))
        c7.append(len(s['PairedAfter'][r]['Data'].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6, c7],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

<H1> Plot histograms of pair distance for all of the studies

In [ ]:
pair_distance = { 'learner': pd.Series(), 'nonlearner' : pd.Series(), 'conditioned' : pd.Series()}

rmax = max(s['PairedAfter'])

for s in paired_studies:
    # Get the pairing for the maximum radius...
    before = s['PairedBefore'][rmax]['Data']
    after = s['PairedAfter'][rmax]['Data']
    t = s['Type']
    pair_distance[t] = pair_distance[t].append(pd.Series(np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)))

data=  [ 
    go.Histogram(x=pair_distance['learner'], name='Learners Radias {0}'.format(rmax)), 
    go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radias {0}'.format(rmax)),
    go.Histogram(x=pair_distance['conditioned'], name='Contitioned Radias {0}'.format(rmax))
]

hist_layout = go.Layout(
    title='Pair Distance Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

<h1> Histogram of pair distances - past a certain pairing threshold 

In [ ]:
thres_pair=20.0 # Threshold for the minimum value to display on the histogram

hist_data_l_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))
hist_data_nl_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))

# Normalized
hist_data_l_N_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")
hist_data_nl_N_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")

hist_layout_Norm = go.Layout(
    title='Pair Distance Histogram Radius Normalized {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig_thres = go.Figure(data=[hist_data_l_thres, hist_data_nl_thres], layout=hist_layout)
hist_fig_Norm_thres = go.Figure(data=[hist_data_l_N_thres, hist_data_nl_N_thres], layout=hist_layout_Norm)

py.offline.iplot(hist_fig_thres, filename='pair-distance_thresholded')
py.offline.iplot(hist_fig_Norm_thres, filename='pair-distance_Norm_thresholded')

<h1> Histogram of intensity ratios

In [ ]:
r  = min(paired_studies[0]['UnpairedBefore'])

core_ratio = { 'learner': pd.Series(), 'nonlearner' : pd.Series(), 'conditioned': pd.Series()}
core = { 'learner': pd.Series(), 'nonlearner' : pd.Series(), 'conditioned': pd.Series()}

for s in paired_studies:
    before = s['PairedBefore'][r]['Data']
    after = s['PairedAfter'][r]['Data']
    t = s['Type']
    core_ratio[t] = core_ratio[t].append(pd.Series(before['core'] / after['core']))
    core[t] = core_ratio[t].append(pd.Series(before['core']))
    core[t] = core_ratio[t].append(pd.Series(after['core']))

data = [
    go.Histogram(x=core_ratio['learner'], name='Learners Radias {0}'.format(r)),
    go.Histogram(x=core_ratio['nonlearner'], name='Nonlearners Radias {0}'.format(r)),
    go.Histogram(x=core_ratio['conditioned'], name='Conditioned Radias {0}'.format(r))
]
    
hist_layout = go.Layout(
    title='Pair Ratio Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='core-intensity')

<H1> General plotting function for synapses

In [ ]:
# Generate shades of blue and orange for the plots and assign colors to the different trace types.
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append('rgb'+str((C[0], C[1], C[2])))
        
    return pl_colorscale

def trace_color(trace):
    global colorindex
    shades = 256
    
    blues_cmap = matplotlib.cm.get_cmap('Blues')
    oranges_cmap = matplotlib.cm.get_cmap('Oranges')

    blues = matplotlib_to_plotly(blues_cmap, shades)
    oranges = matplotlib_to_plotly(oranges_cmap, shades)

    if trace == 'AlignmentPts':
        color = 'black'
    elif 'Before' in trace:
        color = oranges[random.randrange(125, 256)]
    else:
        color = blues[random.randrange(128,256)]
    return color

In [ ]:
def trace_masks(study_map, trace_map) :
    all = OrderedDict()
    before = OrderedDict()
    after = OrderedDict()
    trace = OrderedDict()
    
    # Figure out how many traces we have.
    trace_cnt = 0
    for s, v in study_map.items():
        for t, idx in v.items():
            trace_cnt = max(trace_cnt, idx)
    trace_cnt = trace_cnt + 1
            
    # create mask for each trace
    for t, v in trace_map.items():
        trace_mask = [False] * trace_cnt
        for s, idx in v.items():
            trace_mask[idx] = True 
        trace[t] = trace_mask

    # Create masks for each study.  We want the studies to be in the same order as 
    for s, v in study_map.items():
        all_mask = [False] * trace_cnt
        before_mask = [False] * trace_cnt
        after_mask = [False] * trace_cnt
        for t, idx in v.items():
            all_mask[idx] = True  
            if 'Before' in t:
                before_mask[idx] = True
            elif 'After' in t:
                after_mask[idx] = True           
        all[s] = all_mask
        before[s] = before_mask
        after[s] = after_mask
        
    return { 'all' : all, 'before': before, 'after': after, 'trace' : trace }

def step_buttons(study, masks):
        updatemenus=list([
               dict(
                buttons=list([   
                    dict(
                        args=[{'showlegend' : True}],
                        label='Legend on',
                        method='relayout'
                    ),   
                    dict(
                        args=[{'showlegend': False}],
                        label='Legend off',
                        method='relayout'
                    ),           
                ]),
                direction = 'left',
                showactive = True,
                type = 'buttons',
                xanchor = 'left',
                y = 1.1,
                x = 0,
                yanchor = 'top' 
            ),
            dict(
                buttons=list([   
                    dict(
                        args=[{
                            'visible' : masks['all'][study],
                        }],
                        label='All',
                        method='restyle'
                    ),   
                    dict(
                        args=[{
                            'visible': masks['before'][study], 
                        }],
                        label='Before',
                        method='restyle'
                    ),
                    dict(
                        args=[{
                            'visible' : masks['after'][study], 
                        }],
                        label='After',
                        method='restyle'
                    )             
                ]),
                direction = 'left',
                showactive = True,
                type = 'buttons',
                xanchor = 'left',
                x = 0,
                y = 1.05,
                yanchor = 'top' 
            ),
        ])
        return updatemenus
    
def plot_steps(masks):
    trace = []
    study = []
    # create steps for each trace
    for t, trace_mask in masks['trace'].items() :
        trace.append(dict(label='{0}'.format(t), 
                          args = [{'visible' : trace_mask}], 
                          method = 'restyle'))
    # Now loop over list of study names and create buttons.
    for s, all_mask in masks['all'].items(): 
        study.append(dict(label='{0}'.format(s), 
                        args  = [{ 'visible' : all_mask}, {'updatemenus' : step_buttons(s, masks)}], 
                        method = 'update'))
    return (study, trace)            

def plot_synapses(studylist, 
                  tracelist = ['PairedBefore','PairedAfter','UnpairedBefore','UnpairedAfter'], 
                  slider='Study',
                  radius = False,
                  centroid = False):

    # Change this if you want the point sizes in the plots to be different
    pt_size = 4
    
    # Use the smallest available radius as the default if one is not provided.
    radius = radius if radius else min(studylist[0][tracelist[0]])
    
    if centroid == True:
        centroidlist = []
        for t in tracelist:
            if t != 'AlignmentPts':
                centroidlist.append(t + 'Centroid')
    tracelist = tracelist + centroidlist     
    data = []

    maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
    
    # We would like studies and traces to remain in user specified order....
    study_map = OrderedDict()
    trace_map = OrderedDict()
    for s in studylist:
        study_map[s['Study']] = OrderedDict()
    for t in tracelist:
        trace_map[t] = OrderedDict()
    
    for s in studylist:
        sname = s['Study']
        for idx, t in enumerate(tracelist):
            synapse_data = s[t][radius]['Data']
        
            maxx = max(maxx,synapse_data['x'].max())
            maxy = max(maxy,synapse_data['y'].max())
            maxz = max(maxz,synapse_data['z'].max())
            minx = min(minx,synapse_data['x'].min())
            miny = min(miny,synapse_data['y'].min())
            minz = min(minz,synapse_data['z'].min())
            
            x,y,z = list(synapse_data['x']), list(synapse_data['y']), list(synapse_data['z'])

            color = trace_color(t)
            
            if 'Centroid' in t:
                symbol = 'cross'
                size = pt_size + 4
            elif 'AlignmentPts' in t:
                    symbol = 'square'
                    size = pt_size + 4
            else:
                    symbol = 'circle'
                    size = pt_size
            trace_synapses = go.Scatter3d(
                visible = False,
                x = x, y = y, z = z,
                text = '{0} Synapse'.format(sname),
                name='{0} {1}'.format(t, sname),
                mode='markers',
                marker=dict(
                    symbol = symbol,
                    size = size, color = color,
                    line=dict(color=color, width=0.5), opacity=0.4)
                )
            
            study_map[sname][t] = len(data)
            trace_map[t][sname] = len(data)  
            data.append(trace_synapses)

    layout = go.Layout(
                height=900,
                title='Synapses',
                showlegend = True,
                scene=dict(
                    xaxis=dict(title='X Axis', 
                               range=[minx, maxx], 
                               color='red'),
                    yaxis=dict(title='Y Axis', 
                               range=[miny, maxy], 
                               color = 'green'),
                    zaxis=dict(title='Z Axis', 
                               range=[minz, maxz], 
                               color = 'blue',),
                     
                    aspectmode='cube',
                    dragmode='turntable',
                    
                    annotations = [
                        dict(
                        showarrow = False, text = 'xmin',
                        x = minx, y = (maxy-miny)/2+miny, z = (maxz-minz)/2+minz,
                        ),
                        dict(
                        showarrow = False,
                        x = minx, y = (maxy-miny)/2+miny, z = (maxz-minz)/2+minz,
                        text = "Medial",
                        font = dict(color = "red",size = 12),
                      ), dict(
                        showarrow = False,
                        x = maxx*.95, y = (maxy-miny)/2+miny, z = (maxz-minz)/2+minz,
                        text = "Lateral",
                        font = dict(color = "red",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = miny, z = (maxz-minz)/2+minz,
                        text = "Posterior",
                        opacity = 0.7,
                        font = dict(color = "green",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = maxy, z = (maxz-minz)/2+minz,
                        text = "Anterior",
                        font = dict(color = "green",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = (maxy-miny)/2+miny, z = minz,
                        text = "Dorsal",
                        font = dict(color = "blue",size = 12),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = (maxy-miny)/2+miny, z = maxz,
                        text = "Ventral",
                        font = dict(color = "blue",size = 12),
                      )]
                )
    )

    masks = trace_masks(study_map, trace_map)                
    if slider is False:
        layout['updatemenus'] = step_buttons(studylist[0]['Study'], masks)
        for i in data:
            i['visible'] = True
    else:
        sliders = [dict(
            active = 0,
            currentvalue = {"visible": True},
            len = .9,
        )]             
        (study_steps, trace_steps) = plot_steps(masks)
            
        # Get the mask from the first step, which is in the 2nd element of the args list..
        if slider == 'Study':
            # Make the first study visible
            mask = masks['all'][studylist[0]['Study']]
            sliders[0]['steps'] = study_steps
            layout['updatemenus'] = step_buttons(studylist[0]['Study'], masks)
        else:
            mask = masks['trace'][tracelist[0]]
            sliders[0]['steps']= trace_steps
            
        # Set first step to be visible
        for i in range(len(data)):
            data[i]['visible'] = mask[i]
        layout['sliders'] = sliders
    fig = dict(data=data, layout=layout)
    py.offline.iplot(fig, filename='synapse-plot')
    

<H1> Plot sets of synapses...

In [ ]:
# Get all radii for a specific study


s_names = ['SynStd6025','SynStd8013','SynStd8071','SynStd6791','SynStd7951','SynStd7949','SynStd6493','SynStd7943',
          'SynStd6479','SynStd6481','SynStd6485','SynStd6483', 'SynStd6477', 'SynStdR5Y',
          'SynStdJK6','SynStdJKP','SynStdJKE','SynStdJJY','SynStdJJP','SynStdJJE','SynStdJJ6',
          'SynStdRVA','SynStdRVE','SynStdRVJ','SynStdRVP','SynStdRVT']
# Removed SynStd6475 because the registration of the original image seems incorrect - somehow shifted??
# SynStdJK6 was bounced back on 1/18/18 so take it with a grain of salt for now

#s_names = ['SynStd7947','SynStd7945','SynStd7943','SynStd7941','SynStd6491','SynStd6489'] #Only 3/17/17 Dataset, diff segmenters

s_list = [s for s in paired_studies if s['Study'] in s_names and s['Type'] == 'learner']
# Pick which traces you would like displayed

tracelist =['AlignedUnpairedBefore','AlignedUnpairedAfter','AlignmentPts']
#tracelist = ['PairedBefore','PairedAfter','UnpairedBefore','UnpairedAfter']
#tracelist = ['UnpairedBefore','UnpairedAfter']
print('Plotting {0} studies'.format(len(s_list)))

#Pick the slider mode...

slidermode = 'Study'
#slidermode = 'Trace'
#slidermode = False
plot_synapses(s_list, tracelist, slidermode, centroid=True) 